In [1]:
import sys
sys.path.insert(0, '../BO')

In [2]:
from gp import *
import numpy as np
from sklearn.datasets import fetch_mldata
from BN import *

In [3]:
# Load and format the training and testing data
mnist = fetch_mldata('MNIST original')

In [4]:
BN = BN(mnist)

In [5]:
loss = BN.train_bayesian_nn

In [10]:
bounds = np.array([[-15, 0.0], [5., 150.], [5., 2000.], [1., 5.], [0.01, 10.], [0.01, 10.], [0.01, 10.], [0.01, 10.], [0.01, 10.]])
# maxeps specifies how many BOs we want to run
maxeps = 10
# n_iters is number of iterations we want for each run of BO
n_iters = 100
acqui_eva_num = 5
greater_is_better = False
n_pre_samples = 3
coor_sigma = 1 * np.array([0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2])
burn_in = 20
input_dimension = 9
mode = 'OPT'
acqui_mode = 'OPT'
acqui_sample_num = 3
process_sample_mode = 'normal'
prior_mode ='normal_prior'
likelihood_mode = 'normal_likelihood'

In [11]:
y_list = list()

for i in range(n_iters):
    y_list.append(list())


for j in range(maxeps):
    print ('Running %d episode' % (j + 1))
    xp, yp = bayesian_optimisation(coor_sigma = coor_sigma, 
                                   burn_in = burn_in, 
                                   input_dimension = input_dimension,
                                   n_iters = n_iters, 
                                   sample_loss = loss, 
                                   bounds = bounds, 
                                   n_pre_samples = n_pre_samples, 
                                   acqui_eva_num = acqui_eva_num,  
                                   greater_is_better=greater_is_better, 
                                   mode = mode, 
                                   acqui_mode = acqui_mode, 
                                   acqui_sample_num = acqui_sample_num,   
                                   process_sample_mode = process_sample_mode, 
                                   prior_mode = prior_mode, 
                                   likelihood_mode = likelihood_mode)
    y_train = yp[n_pre_samples:]
    for idx in range(len(y_train)):
        if idx == 0:
            y_list[idx].append(y_train[idx])
        else:
            y_list[idx].append(np.min(y_train[:idx]))

Running 1 episode
Start presampling...
	Learning rate: 0.0007649861466, training epochs: 44, batch size: 1673, n_samples: 2, prior_sigma: 5.12836, init_sigma_weights_mu:  3.15833, init_sigma_biases_mu: 1.88039, init_sigma_weights_rho: 8.14484, init_sigma_biases_rho: 2.53966
Accuracy 0.0972380952381
	Learning rate: 5.16608751269e-06, training epochs: 144, batch size: 310, n_samples: 3, prior_sigma: 9.03428, init_sigma_weights_mu:  9.26899, init_sigma_biases_mu: 6.27637, init_sigma_weights_rho: 7.83383, init_sigma_biases_rho: 9.04714


KeyboardInterrupt: 

In [4]:
def return_mean_std(y_list):
    mean_list = list()
    std_list = list()
    for one_list in y_list:
        mean_list.append(np.mean(one_list))
        std_list.append(np.std(one_list))
    return mean_list, std_list
mean_list, std_list = return_mean_std(y_list)

In [ ]:
import matplotlib.pyplot as plt
x = [i+1 for i in range(len(y_list))]

plt.errorbar(x, mean_list, yerr = std_list, fmt = '-')

plt.show()

In [11]:
import pickle
opt_dict = {}
opt_dict['y_list'] = y_list
with open('BN_MCMC_OPT.pkl','wb') as file:
    pickle.dump(opt_dict, file)

In [15]:
with open('BN_MCMC_OPT.pkl', 'rb') as pickle_file:
    data1 = pickle.load(pickle_file)